In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re
import time
import pickle

In [2]:
data=pd.read_csv("../input/records/records.csv")
all_text=data["transcription_text"]

In [3]:
data.shape

(35168, 16)

In [4]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()
stop_words.remove("of")
punctuation
punct='!"#$%&\'()*+-/<=>?@[\\]^_`{|}~'
def process(s):
    s= re.sub("([\(\[]).*?([\)\]])"," ",s)
    for p in punct:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] #optional
    return s
start=time.time()
for i in tqdm(range(35168)):
    all_text[i] = process(all_text[i])
for i in tqdm(range(35168)):
    all_text[i] = " ".join(all_text[i])
end=time.time()
print(end-start)

100%|██████████| 35168/35168 [00:18<00:00, 1872.39it/s]

111.99071741104126


In [5]:
all_text

0        one hundred dollars . I never heard Bagby comp...
1        The Freedmens Bureau vs. B.M . Richardson Assa...
2        The Freedmens Bureau vs. B. M. Richardson , Co...
3        one hundred dollars , two suits of clothes wou...
4        questions , I gave answer , I whipped.— All I ...
                               ...                        
35163    648 Z Date.Name of Male.Place of Residence.Nam...
35164    650 Date . Name of Male . Name of Female . Pla...
35165    Sarah Brame Died June 25th 1847 . 1866 1847 De...
35166    Frances S. Benners Born August 30th 1795 Died ...
35167    State of North Carolina Bladen County Alfred ....
Name: transcription_text, Length: 35168, dtype: object

****

****

In [6]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 269.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=e6addb2bfc07bca908eec5d980703921bdf313aa050d36237ed784d8b55125ba
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
Note: you may need to restart the kernel to use updated packages.


In [7]:
def num_of_element(clust_assign, clus_sent) :
    dic={}
    L=list(clust_assign)
    for i in range(len(clus_sent )):
        dic[i]=L.count(i)
    
    return dic
        

In [8]:
def Max_clus_size(list1, N):
    _list = []
  
    for i in range(0, N): 
        max1 = 0
          
        for j in range(len(list1)):     
            if list1[j] > max1:
                max1 = list1[j];
                  
        list1.remove(max1);
        _list.append(max1)
          
    print(_list)

In [9]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(all_text)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/1099 [00:00<?, ?it/s]

In [10]:
embeddings.shape

(35168, 384)

In [11]:
start=time.time()
# Normalize the embeddings to unit length
embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=3.5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

242.45592546463013


In [12]:
len(clustered_sentences)

309

In [13]:
cluster_assignment

array([ 58,  58,  58, ..., 125, 125, 186])

In [14]:
dic1=num_of_element(cluster_assignment,clustered_sentences)

In [15]:
dic1[0]

153

In [16]:
len(clustered_sentences[250])

66

In [17]:
clustered_sentences[0][:15]

['Charges Specifications preferred Second Lieutenant James H. Griggs Thirty Seventh Regiment United States Colored Troops . Charge Misapplication of public money intrusted payment of men command . Specification 1st In Second Lieutenant James H. Griggs . Thirty Seventh Regiment United States Colored Troops , officer Service of United States Acting Assistant Quartermaster Post of Fort Macon North Carolina , command one Bonaparte Fulford citizen employment of United States first fifth day of July A.D. 1866 , first twentieth day of August A.D. 1866 , inclusion compensation of forty dollars per month , furnished Brevet Lieutenant Colonel P.D . Stubbs Captain Assistant Quartermaster United States Volunteers , Cheif Quartermaster Military Commander of North Carolina twelfth day of November A.D. 1866 , check number payable order Assistant Treasury New York sum of thirty three dollars thirty two cents purpose of paying said Bonaparte Fulford',
 'lawfull wages period herein Specified withold fai

In [18]:
clustered_sentences[300][:50]

['No 22 A Miss Alice Ashley per Gabriel Davis Attorney I.B . 36 A ACSC GA 65 Applies restoration of property Decb 9 65 Restored',
 'A 29 Alston W.A Application Plantation Georgtn Dist A 29 Hd Qr Acty Sub Asst Comr Bureau of R.F . AL . Georgetown S.C. Feby 8th 1866 Respectfully forwarded recommended B.F. Smith Bvt Lt Col U.S.A. Actg Sub Asst Commr B of R.F.AL . Feby 20 66 Restored',
 'B No 19 W Baggett James . W. Charleston So Ca Applies Restoration of cotton plantation Prince Williams Parish Beaufort Dist So . Ca . property I.B . 52 B Asst Com SC Ga 65 Thro William Whaley Esqre of Charleston So Ca Oct 11 1815 order issued restoration Restored R.S . Recd AAGO Asst Com SC Ga Oct 1165',
 'No : 15 B Daniel Blake Application Restoration of Real Personal Property . Palmetto Hill Combahee River I.B . 41 B Asst Com SC Ga 65 Restored RS Oct 6 , 1865 . Order issued restoration of property . Recd AAGO Asst Com SC Ga Oct 5 , 1865',
 'No 11 B Daniel Blake Application Restoration of Real Personal Pr

In [19]:
clustered_sentences[200][:]

['Bastrop Bastrop Co. Texas , March 28 , 1867 I F. Laake , agent trustees of church building owned German Methodist Episcopal Church South , Bastrop Texas , hereby agree contract rent said building school purposes , Bureau Refugees Freedmen Abandoned Lands , State of Texas , three months , beginning 1st day of April 1867 ending 30th day of June 1867 , monthly rental of fifteen Dollars currency , paid monthly ; Bu . of R.F . A.L . agreeing become responsible damage scholars may said building enclosure time Frany Laake Witness Geo . M. Decherd To Supt . of Schools Bu . R.F . A.L State of Texas',
 'O.S . F.J. Reed . June 767P.U . Bastrop , June 367 Porter Byron Sub Asst Comr Enclose Miss M E Wilder School report May 1867 . States Soloman Fish resident of Sinbin Bastrop Co agreed give lot of ground timber enough put school House , The Lot given freedman corporation of Bastrop selected of stone foundation ground hopes open School next October , Will ask assistance exhausted extracted , Cold

In [20]:
clustered_sentences[250][:]

['2 Complaints 1868 Date County Names of Parties Notice of Complaint Action Taken Remarks Jany 8 Franklin Geo N Macon vs Frank . Bob Ned Dissatisfaction settlement of ac , crop made 1867 Jany 868 ac settled See T.R . p 66 Jany 8 Franklin Bill Guplin vs Joe Foster Deft took carried away 2 bbls of corn Feby67 . Falsely claiming plff owed Jany 868 . Plff requests proceeding stayed Court Jany 9 Granville DE I. Baskerville vs Aaron Joe Lyne Refusal of Defts certain work expressly stipulated contract 1867 Summons issued returnable Jany 27 11 Judgment Jany 27 Deft killed plffs steer July67 hard driving business Deft pay 10 damages Jany 9 Granville James A Bullock vs William Jeffreys Refusal strip tobacco made 1867 Summons issued returnable Jany 27 11 No appearance Jany 9 Franklin Bob Hunt vs Frank Foster F Deft hired Plffs daughter year 1867 refuses pay amt 25 Jany 968 . On settlement of acs Mason Frank others bal due Frank 15.27 . Ordered Mason pay amt Bob , 7 children , Frank none See L.B .

****

# all_mpnet_base_v2 , threshold=3.5

In [21]:
model_1 = SentenceTransformer('all-mpnet-base-v2')
embeddingss = model_1.encode(all_text)
embeddingss.shape

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/1099 [00:00<?, ?it/s]

(35168, 768)

In [22]:
embeddingss.shape

(35168, 768)

In [23]:
start=time.time()
# Normalize the embeddings to unit length
embeddingss = embeddingss /  np.linalg.norm(embeddingss, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=3.5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddingss)
cluster_assignment_1 = clustering_model.labels_

clustered_sentences_1 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment_1):
    if cluster_id not in clustered_sentences_1:
        clustered_sentences_1[cluster_id] = []

    clustered_sentences_1[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

359.73620104789734


In [24]:
len(clustered_sentences_1)

245

In [25]:
dic_1=num_of_element(cluster_assignment_1,clustered_sentences_1)

In [26]:
cluster_assignment_1

array([ 53, 140, 140, ..., 179,  24, 147])

In [27]:
len(clustered_sentences_1[180])

22

In [28]:
clustered_sentences_1[0][:]

['B . North Carolina Columbus County This day came Calvin E. Gore , justice of Peace said County . William Q. Maultsby , sworn saith oath well acquainted Ben Simpson freedman , known past seven eight years , always bad character , generally regard person of untruth , vicious disposition , could believe oath , would regard dangerous character community . He swears , known Benjamin M. Richardson six years , known peaceable quiet moral man , of kind disposition , employ time negroes slavery , freedmen service since liberation , always treated uniform kindness ; regarded community worthy citizen . He swears Mr Richardson politics opposed war , took active part encouraging rebellion , influence discourage ; war man ; point of humanity helped poor needy , without regard race color W. Q. Maultsby seal Signed Sealed Sworn This March 19th 1866 , Calvin C. Gore J.P. seal',
 'C North Carolina Columbus County This day came Calvin G. Gore , justice of peace said County . J Lloyd McKay sworn saith o

In [29]:
clustered_sentences_1[200][:20]

['NARA 128 98 No . 24 LEASE FOR HOUSES AND TENEMENTS . Memorandum of Agreement made 12th day of April 1865 , D. Heaton Supg . Special Agent of Treasury Department , duly appointed Acts of Congress respectively approved March 12 , 1863 , July 2 , 1864 , taking charge of captured abandoned property , leasing abandoned confiscable lands , houses , tenements Sixth Special Agency , Mrs. Mary R. Hooks of Wilmington County of New Hanover State of North Carolina WITNESSETH , pursuance of said Acts , of instructions of Secretary of Treasury , said Agent , behalf of United States , agrees upon terms hereinafter contained , lease said Mrs. Mary R. Hooks First day of April , 1865 , month month , either party hereto liberty terminate lease end of month date hereof , following described premises : House Cor . of Mulberry Third Sts . Block 205 Wilmington North Carolina And said Mrs. Mary R. Hooks hereby agrees pay said Agent One dollars per month , month date hereof , long shall continue possession o

In [30]:
clustered_sentences_1[150][:20]

['AsstSUBASSISTANT COMMISSIONERS MONTHLY REPORT Education of Freedmen Refugees SubDistrict , State of La charge of Month of Febry 1868 , 1 . Name of SubDistrict Avoyelles 2 . Whole number of Refugee Freedmens Schools District 3 Day 3 Night Sabbath 3 . Location of Schools Marksville Hydropolis Holmesville 4 . Whole number of Teachers 3 White 1 Colored 2 5 . Names postoffice address of DaySchool Teachers A Hitten Holmesville La Miss Tassin Marksville La 6 . Whole number of Schoolhouses Freedmen District Their condition , capacity , value , owned In Marksville . Mrs. Plumel Hydropolis Holmesville Houses furnished used ordinary Quarters 7 . Number of visits Schools 3 Day 3 Night Sabbath 8 . Number of educational meetings held month 1 Where Marksville 9 . Number names of places , destitute , DaySchools might organized none , teacher supported Mansura Miroville 10 . Number of pupils would attend Schools fifty 11 . Amount would probably raised Freedmen , school purposes , destitute neighborho

In [31]:
clustered_sentences_1[180][:]

['St Johns Nov .',
 'St James No.9 Nov',
 'St John No3 Nov',
 'St James No 2 Nov',
 'St John No 5 Nov',
 'St John No 4 Nov .',
 'St James No 13 Nov .',
 'St James No.1 Nov',
 'St James No.6 Nov',
 'St John No Dec .',
 'St. John No5 Dec ,',
 'St James No.2 Dec .',
 'St. James No 9 Dec .',
 'St James No.6 Dec .',
 'St James No.1 Dec .',
 'St John No.4 Dec',
 'St John No.3 Dec',
 'St James No 3 Dec 1st 15th Dec',
 'St John No Dec',
 'St John No 3 Dec',
 'St James No 2 Dec',
 'St James No 3 Dec']

****

# all-MiniLM-L6-v2 , threshold=2

In [32]:
start=time.time()
# Normalize the embeddings to unit length
embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=2) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddings)
cluster_assignment2 = clustering_model.labels_

clustered_sentences2 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment2):
    if cluster_id not in clustered_sentences2:
        clustered_sentences2[cluster_id] = []

    clustered_sentences2[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

240.66173148155212


In [33]:
len(clustered_sentences2)

1008

In [34]:
dic2=num_of_element(cluster_assignment2,clustered_sentences2)

In [35]:
dic2[0]

307

In [36]:
len(clustered_sentences2[1007])

31

In [37]:
clustered_sentences2[0][:10]

['L.W . Stevenson , Supt of Education I would observe information desire remain division of county yr. Am near Seven months teaching , affected , respectable improvement among Pupils . Respectfully H. Ames ED FORM , NO . 3 TEACHER ’ S MONTHLY SCHOOL REPORT For month of May , 1870 District of County , Guadaloupe State , Tex Name of School , High Hill Grammar School Name of Teacher , Hugh Ames Forwarded Mr. Andrew Wade , RECD OFFICE B.R.F . A.L . RECD JUN 5 1870 Explanations If of within questions answered Teacher , may left Superintendent . Place answer , extent of knowledge opposite question , fill remaining spaces . Night Sabbath Schools taught group of Teachers reported . In Remarks notice important fact blank call . It hoped Educational Societies adopt form of Report ; , Bureau furnish blanks . Should extended forms used , would oblige Commissioner include items of . Bureau R.F . A.L Washington . D.C.. October 1 . 1868',
 'Teachers Monthly School Report . For month of March 1866 . D

In [38]:
clustered_sentences2[1][:20]

['Invoice of Tuition Tickets sold Anora Killgrove , Principal Teacher Abraham Lincoln School , 1st 25th day of July 1866 , transfer of blank tickets cash arising sale of tickets E.S . Stoddard Director . DATE TICKETS . DOLLS . CTS . July 1st Total amount of blank tickets received per invoice ... 100 July 12 Sold 1.50 ... 2 3 00 Sold 1.00 ... July 25 Sold less month , pro rata ... Entered folio 198 Tuition Ledger Total 2 3 00 RECAPITULATION Total amount of tickets sold 1.50 ... 76 117 00 Total amount of tickets sold 1.25 ... 7 8 75 Total amount of tickets sold 1.00 ... Total amount of tickets sold less month , pro rata Total tickets sold cash recd 83 122 754 BALANCE 17 I HEREBY CERTIFY , I 25th day of July New Orleans La . transferred E.S . Stoddard Director , amount of blank tickets cash stated . Anora Killgrove Principal . I hereby acknowledge I 25th day of July 1866 , NEW ORLEANS , LA. , received Anora Killgrove Principal of A Lincoln School , Three Dollars Cents cash seventeen unsol

In [39]:
clustered_sentences2[1000][:]

['BUR . REF. , FREEDMEN AND ABANDONED LANDS , State of Texas . SCHOOL REPORTS FOR THE MONTH OF January 1866 .',
 'BUR . REF. , FREEDMEN AND ABANDONED LANDS , State of Texas . SCHOOL REPORTS FOR THE MONTH OF February 1866 .',
 'BUR . REF. , FREEDMEN AND ABANDONED LANDS State of Texas . SCHOOL REPORTS FOR THE MONTH OF March 1866 .',
 'BUR . REF. , FREEDMEN AND ABANDONED LANDS State of Texas . SCHOOL REPORTS FOR THE MONTH OF April 1866 .',
 'BUR . REF. , FREEDMEN AND ABANDONED LANDS State of Texas . SCHOOL REPORTS FOR THE MONTH OF May 1866 .',
 'BUR . REF. , FREEDMEN AND ABANDONED LANDS State of Texas . SCHOOL REPORTS FOR THE MONTH OF June 1866 .',
 'BUR . REF. , FREEDMEN AND ABANDONED LANDS , State of Texas . SCHOOL REPORTS FOR THE MONTH OF July , 1866 .',
 'BUR . REF. , FREEDMEN AND ABANDONDED LANDS State of Texas . SCHOOL REPORTS FOR THE MONTH OF AUGUST , 1866 .',
 'BUR . REF. , FREEDMEN AND ABANDONED LANDS , State of Texas . SCHOOL REPORTS FOR THE MONTH OF September , 1866 .',
 'BUR .

In [40]:
clustered_sentences2[1005][:]

['Bureau of Refugees Freedmen Abandoned Lands Office Sub Dist of Oxford Oxford N.C. May 2nd 66 Nat Nicholson Pltf . vs. S S Lucy Russel Deft . Pltf . Complains wife , Tempy , daughter Martha Ann , worked Mrs. Lucy Russel , parts of summer of 1865 never paid said labor . Nat Nicholson duly sworn states follows , wit , That commenced work Mrs Russel , Deft , 1st day of May 65 worked fieldhand untill 7th day of July 65 , wife work servant Deft . 1st of May 65 untill 2nd Sept 65 child aged 13 yrs worked servent Deft . 1st of May untill 2nd Sept 65 , labor received pay , Nicholson states stoped work 7th of July 65 consent of Mrs Lucy Russel , wife child left Mrs Lucy Russels service consent order , Nicholson , sined house of Deft . wife child , boarding assisting boarding family , untill 2nd Sept 65 .',
 'Bureau of Refugees Freedmen A. Lands Plymouth D 136 Captain I wish call attention action of Lieut Swarzt absence Sold Cotton corn regardless of price amt near I Learn of four five hundred 

****

# all_mpnet_base_v2 , threshold=2

In [41]:

start=time.time()
# Normalize the embeddings to unit length
embeddingss = embeddingss /  np.linalg.norm(embeddingss, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=2) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddingss)
cluster_assignment_2 = clustering_model.labels_

clustered_sentences_2 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment_2):
    if cluster_id not in clustered_sentences_2:
        clustered_sentences_2[cluster_id] = []

    clustered_sentences_2[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)


360.5178802013397


In [42]:
len(clustered_sentences_2)

752

In [43]:
dic_2=num_of_element(cluster_assignment_2,clustered_sentences_2)
l_2=[dic_2[i] for i in range(len(clustered_sentences_2))]
maxx=np.max(l_2)
minn=np.min(l_2)
maxx, minn

(369, 5)

In [44]:
l_2.index(35)

22

In [45]:
len(clustered_sentences_2[22])

35

In [46]:
clustered_sentences_2[69]

['Miss Delia Banner Report Nov. 15th . 1865',
 'Delia Banner Depart Nov. 1 Nov. 15th 1865',
 'Delia Banners Report From Nov. 15 Nov. 30th 1865',
 'Mrs. Delia Banners Report Oct. 30th . 1865 Delia Banners Report of No . of Pupils Nov. 1865',
 'School Report . Plaguemine , Iberville Par . La . June May 1866',
 'School Report Liberty School . Par . of Pointe Coupee . July 1866 . Simpson E. W. Princ .',
 'School Report Par . of Pointe Coupee sch . Par , of Pointe Coupee . July 1866 . Simpson . E. N. Princ . Baptiste Le Jeune Louis Panche Theodore CheneVere Lanalois Clere Cassaivoux',
 'School Report of Liberty School Par . of Pointe Coupee . July 1866 Wade , Eloise B. Princ .',
 'Plaquemine School Report July 25 1866',
 'School Report of Liberty School Par . of Pointe Coupee . August 66 Wade , Eloise B .',
 'Report of Anna M Duncan Plaquemine , Par . of Iberville Oct. 29 1866',
 'School Report of Duncan , Anna M. Par . of Iberville La . Nov. 66 .',
 'Report of Anna M. Duncan , Par . of Ibe

In [47]:
clustered_sentences_2[260]

['SCHOOL REPORT Of Watt Bonner . County of Colorado Co FOr Month of January , 1867',
 'SCHOOL REPORT Of Watt Bonner County of Colorado Co. Tex . For Month of February , 1867',
 'SCHOOL REPORT OF Watt Bonner COUNTY OF Colorado Texas For Month of March 1867 . FORWARDED April 1st 1867 . RECEIVED O.S . April 3d 1867 . Watt Bonner . Columbus . Texas . School Report March 1867 .',
 'SCHOOL REPORT Of Edward Henry Adams County of Colorado Cog . For month of March 1867 . Forwarded April 8th 1867 . Received O.S . April 12th 1867 . Mr E. H. Adams . Colorado Co. Texas . School Report March 1867 .',
 'SCHOOL REPORT OF L.P. Wheeler COUNTY OF Colorado For Month of April 1867 . FORWARDED May 6th 1867 . RECEIVED O.S . May 8th 1867 .',
 'SCHOOL REPORT OF Edward H. Adams COUNTY OF Colorado Tex For Month of April 1867 . FORWARDED May 6th 1867 . RECEIVED O.S . May 8 1867 . Mr E.H. Adams . Colorado Co. Texas . School Report April . 1867 .',
 'SCHOOL REPORT OF Adela L Chittenden COUNTY OF Colorado For Month 

In [48]:
clustered_sentences_2[479]

['School Report of Forrest Hooks County of Bowie , State of Texas For month of March The school open 15th of March 1868 , Forwarded April 4th 1868 Received 0.S . 1867 .',
 '143 EB Vol 1 SCHOOL REPORT OF Forrest Hooks . COUNTY OF Bowie State of Texas For month of April 30th . FORWARDED May 186 7 8 RECEIVED O.S.A.C . May 2 186 7 8 Forwarded May 2nd 1868',
 'SCHOOL REPORT OF Forrest Hooks . COUNTY OF Bowie . State of Texas For month of April 30th . FORWARDED 1867 RECEIVED O.S . ... ... ... 1867 .',
 'SCHOOL REPORT OF Forrest Hooks , County of Bowie . For month of May . Forwarded July 1 1868 . Received O.s . ... 186 .',
 'SCHOOL REPORT Of Forrest Hooks . County of Bowie For month of June . Forwarded July 4 . 1868 . Received O.S.A.C . July 4th 1868 .',
 'School Report Of Forrest Hooks . County of Bowie For month of July . Forwarded August 1st 1868 . Received O.S . AC July 1st 1868 . Forwarded O.S.A.C . July 1st . 1868',
 '194 L.R.B . 1868.4 195 E. M.B 1868 . SCHOOL REPORT Of Forrest Hooks .

In [49]:
len(clustered_sentences_2[705])

96

**good one**

In [50]:
clustered_sentences_2[47][:5]

['REPORT OF SCHOOL FOR FREEDMEN In Richmond Ft Bend Co , Texas , For Month Ending Feb 15th , 1866 DAY SCHOOL Total No . Pupils last Rep.Admitted.Left.Total Attendance.Average Attendance.Alphabet.Spelling.Reading.Mental Arithmetic.Written Arithmetic.Writing Copy Book.Writing Slate Blk.Bd.Geography.Grammar . BOYS , none3533525151500001000 GIRLS , none252253020100000700 TOTAL , 6056055352500001700 NIGHT SCHOOL Total No . Pupils last Rep.Admitted.Left.Total Attendance.Average Attendance.Alphabet.Spelling.Reading.Mental Arithmetic.Written Arithmetic.Writing Copy Book.Writing Slate Blk.Bd.Geography.Grammar . MALES , none250252020100000700 FEMALES , none290292010140000800 TOTAL , 5405440302400001500 SUNDAY SCHOOL Total attendance of Adults , 40 Total attendance of Children , 30 I certify Report correct , Regulations complied . C.E . Coleman , Teacher',
 'REPORT OF SCHOOL FOR FREEDMEN In Richmond Ft Bend Co , Texas , For Month Ending March 15th , 1866 DAY SCHOOL Total No . Pupils last Rep.Admi

**Great one**

In [51]:
clustered_sentences_2[632]

['6 Section IV . This rule shall standing fundamental rule . RULE II . OFFICERS AND TERM OF OFFICE Section I . The officers of Corporation shall President , Secretary , Treasurer . They shall elected passage of rules ; afterwards , annual meeting , severally continue office successor duly elected enters upon duties of office . EXECUTIVE COMMITTEE . Section II . An Executive Committee , consist of five members , shall elected time , hold office said term . The President shall , ex officio , member of Committee Chairman . VACANCIESHOW SUPPLIED . Section III . In case of mentioned officers members of committees shall removed office expiration of term , vacancy may filled election meeting of corporation , person elected shall serve unexpired term of predecessor . RULE III . DUTIES OF PRESIDENT . Section I . The President shall preside meetings of Corporation of Executive Com 7 mittee , shall conduct things agreeably customary rules of deliberative bodies ; absence , chairman shall chosen o

In [52]:
clustered_sentences_2[0][10:50]

['Samuel Phillips acct J.D . Wadkins For three months service promised pay 50cts per day Samuel Phillips I certify Samuel Phillips sworn acct 26th day of Feb 1868 J S Johnston JP',
 'John P William 1868 In ac May 1st Edward Wharton To 4 months labor c14 36.00 1869 JP Williams J S Williams To Bal due last years labor 74 00 130.00 Capt Hildebrandt please give J W Thompson order collection of amount I need of Money , Mr Jno . P. Williams died estate insolvent property enough satisfy claim Edward x mark Wharton Witness S. B. Smith',
 'Contract agreement made R. M. Abbott , Laborers . Rockingham County .',
 'Mr Powhatan McCollum Dr To Jane Lemons year 1865 Self two Children Surrender 1st Decr following making Seven Months To services self cooking 7 Mo 4 pr . Month 28 00 To services of Daughter Rhoda 7 Mo 4 28 00 To services of Son 7 Mo 2 pr . Mo . 14 00 60.00 Cn By 6½ Barells Corn 5 pr . Bl . 32 50 By 3 lb Wool 50 cents 1 50 By 3 Gal Sorgum 50 ct 1 50 By 1 small Pig weigh 20 lb 1 00 1 00 By

In [53]:
clustered_sentences_2[500][:]

['Colored APPLICATION OF John Pearson Fanny Avery Dick Avery State of North Carolina Morganton June 15 1867 .',
 'Colored APPLICATION OF Clarissa Sudreth Kate Caldwell State of North Carolina Morganton June , 15 , 1867 .',
 'Colored APPLICATION OF Delia Scott Luncenda Walton State of North Carolina Morganton June 17 , 1867 .',
 'Colored APPLICATION OF Wash Johnson State of North Carolina Morgantin June 17 , 1867 .',
 'Colored APPLICATION OF Clarra Erwin Nathanal , Avery Hannah Gaylord State of North Carolina Morganton June 17 1867',
 'Colored APPLICATION OF Elea Smith State of North Carolina Morganten June 18 1867 .',
 'Colored Application of Lukey Pearson Nancey Hanson , Dinah Avery State of North Carolina Morganton June 23rd 1867 .',
 'Colored APPLICATION OF Wm Edwards George Presnell Pink Mc Entyre State of North Carolina Morganton June 24 1867 .',
 'Colored Application of Sandey Erwin George Pearson Chaney Tate State of North Carolina Morganton , June 25th 1867 .',
 'Colored Applic

In [54]:
clustered_sentences_2[501][:]

['O.S.F.S . Recvd Apl 2967 Palestine Apl 1067 Morrison J.H . S.A. Com Report Freedmen want of schools recommended Mr. Boyd teacher',
 'Palestine Apl 1067 J T Kirkman Lt 26 US In A A A Genl Supt of S Reports freedmen anxious school : one needed Palestine recommend Mr Pat Boyd Teacher . JH Morrison Sub Asst Com',
 'B.R.F . A.L . M33 May 1467 Palestine May 767 Morrison I.H . Sub Asst Com Reports one school Palestine children of Freedman of labor successful operation 68 scolhars . A suitable house purchased Freedman 600 . 200 paid paid 1st of Dec 1867 . The Colored people generally county subject of schools earnest consideration , determined take steps . School opened May 4th Office May 1467',
 'Office sub Asst Comr B.R.F A.L . Palestine May 7th 1867 Sir I report one school Palestine children of Freed people of Color successful operation fiftyeight schollars , A suitable house purchased Freedmen , six hundred dollars . Two hundred dollars paid paid 1st day of December 1867 . The colored pe

In [55]:
clustered_sentences_2[200][:]

['M 109 State of Texas , County of Smith Know men presents Joseph Smith of aforesaid State County of first part , Bvt Lieut Col D L Montgomery , Sub Asst . Com . B R F A.L . State of Texas , of Second part , hereby make enter following agreement . Witnesseth That I Joseph Smith of first part hereby agree give immediate possession of certain frame house situated South side of Public Square of Town of Tyler Smith Co. joining old Confederate drug store Sub Assistant Com . B.R.F A.L . school purposes , freedpeople consideration of payment of monthly rent of Seven dollars US . currency . That I , D.L . Montgomery , Bvt Lt Col U.S.A. Sub Asst Com . B.R.F . A Lands State of Texas of first part hereby agree Agent of United States Government pay Joseph Smith of second part owner proprietor of aforesaid building monthly rent of seven dollars US Currency . In witness whereof herewith subscribe names twelfth day of October 1867 J P. Smith D. L. Montgomery Bvt . Lt. Col US.A . Sub Asst . Com . Sign

In [56]:
clustered_sentences_2[700][:]

['SCHOOL REPORT Of San Antonio County of Bexar For Month of November , 1866',
 'School Report Of San Antonio County of Bexar For Month of Jan , 1867',
 'SCHOOL REPORT OF San Antonio COUNTY OF Bexar For Month of March 1867 . FORWARDED March 31 1867 . RECEIVED O.S . April 5th 1867 . Mr N. Whiting . San Antonio . Texas . School Report March . 1867 .',
 'SCHOOL REPORT Of San Antonio County of Bexar For Month of April 1867 Forwarded Apr 30 1867 . Received O.S . May 7th 1867 . Mr N. Whiting . San Antonio , Texas . School Report April , 1867 .',
 'W7BRc . S.a1867 . SCHOOL REPORT of San Antonio County of Bexar For Month of April 1867 . FORWARDED Apr 30 1867 . Recd O.S.A.C San Antonio May 1 , 1867 . RECEIVED O.S . ... ..1867 .',
 'SCHOOL REPORT OF San Antonio COUNTY OF Bexar For Month of June 1867 . FORWARDED July 1 1867 . RECEIVED O.S . July 10 1867 .',
 'SCHOOL REPORT OF Beaumont COUNTY OF Bexar For month of August FORWARDED September 2nd 1867 .1868 RECEIVED O.S . ... ... ... . 1867 .1868',
 

In [57]:
clustered_sentences_2[701][:]

['Sept 30 . 1867 I Resived Part of Tooles One Part of vise Anvill Screw Plte one hand hammer found George W Stuard Pessions Kernersville NC John Jessop',
 'Capt . Hay Sept 10th 67 Dear sir : I paid Josip Noblin 50.00 Repairing Buggy Respectfuly F.B . Hester work dun . pool 1865 Sept 1067 certify buggy first old work F.B . Hester',
 'Shop ac 1867 H T. Royster Norvell Lyne 17.08 12.55 29.63 15.85 45 48 30 15.48',
 'ac H. J. Royster Norvel Lynes 1866',
 'Shop AC Bullock Royster 1867 To Norvell Lyne',
 'Report J.H . Ford Feby 6th 1865 Report Mr. Wheelock Supt of Carpentershop please make repairs . E. M. Wheelock',
 'March 10th 67 MD Royster Dr To James Overby Adm of Lewis Davis 7th . Dec 11.50 repair of wagon J . Black receipt Lewis Davis told owe Thirty two dollars M S Thomas Jim Overbys Acct .',
 'ac R Bulloch With Norvel Lynes 1866',
 'AC Richard Bullock With Norvell Lynes 1866',
 'Millersville NC Reuben Hodge 1865 To Robin Hinton dr To Repairing 4 wagon wheels 1150',
 '1867 John Stills

**good**

In [58]:
clustered_sentences_2[702][30:]

['Charleston SC Sept 5th 1866 Lieut A.D.C . Sir I honor respectfully request restoration of tract of land Daniels Island , St. Thomas Parish , belonging Estate of John Jenkins , property , time back possession of Freedmen , squatted upon place , since permission cultivated express provision part take care of property , remain present crop gathered . The said land property of Sisters . I honor Yours Respectfully Daniel Jenkins We undersigned hereby certify land situated St. Thomas Parish property of Daniel Jenkins Sisters Jno . King F L Serjeant Sworn 5th day of Sept. 1866 W. C. Finley Notary Public',
 'Carolina Ex parte Admix Petition restoration of Lot Beaufort ENCLOSURES . Restored Jan 7 , 1867 Hdqrs . Asst . Com . December 30th 1866 . Headquarters , Assistant Commissioner , BUREAU REFUGEES , FREEDMEN AND ABANDONED LANDS , South Carolina . Charleston , S.C. , December 31st 1866 . I.B . 42 J Respectfully referred Mr. H. W. Smith Agent Bureau RF AL make necessary restoration papers cas

**yup**

In [59]:
clustered_sentences_2[704][:]

['O.S.F.S . Recd . July 267 B29 Huntsville June 3067 Butler James P. Sub . Asst . Com . School report month of June',
 'O.S.F.S . Recd June 767B22 Huntsville June 167 Butler James P Sub Asst Com Report Houses hired school purposes Three Schools control of Bureau Sub Dist',
 'O.S.F.S . Recd June 867 Huntsville June Butler James . P. Sub Asst . Com Report Schools',
 'O.S.F.S . Recd . July 267 B29 Huntsville June 3067 Butler James P. Sub . Asst . Com . School report month of June',
 'O.S.F.S . July 1167 B.31 . Huntsville June 30 1867 Butler James P. Sub Asst Com Reports number of Schools operation Sub Dist .',
 'O.S.F.S . Rec Aug 8 67 B.35 Huntsville July 3167 Butler Jas . P. S.A.C . Reports schools July',
 'O.S.F.S Recd B,40 Huntsville July 3167 Butler James P. Reports schools closed present . good building nearly completed ready opened Sept 1st File 6',
 'Office S.A.C B.R F c Huntsville July 31 . 1867 Lt J T Kirkman A.A.A.G . In regard subject of education , present schools operation , 

In [60]:
clustered_sentences_2[705][:]

['TEACHERS MONTHLY SCHOOL REPORT , For Month of December 1866 . A School district control of one Teacher , Teacher one assistant , reported one School . To forwarded soon possible 1st of month . Number of DaySchools ... . 1 Number of NightSchools ... ... . 1 Location , Name of School ... . Bullitts Bayou Colony When opened ... . 1st Jan 1866 Societies , c. , Patrons ... . Number of Schools sustained freedman ... . 1 Number of Schools sustained part freedmen ... . Number of teachers transported Bureau ... . Number of School buildings owned freedman ... . 1 Number of School buildings furnished Bureau ... . Whole number of teachers ... . ... . White ... . Colored.. 1 ... . Whole number of pupils enrolled ... . 25 ... . Male . 8 ... . Female 18 ... . Number of pupils enrolled last report ... . 25 Number left school month ... . . Number of new scholars month ... . 4 Average attendance ... .. 25 Number of pupils paying tuition ... . 25 Number of White pupils ... .. Number always present ... 

In [61]:
clustered_sentences_2[706][:]

['SUBASSISTANT COMMISSIONERS MONTHLY REPORT Education of Freedmen Refugees , SubDistrict , State of charge of Month of 1868 , 1 . Name of SubDistrict I give I Cabarrus Co. NC 2 . Whole number of Refugee Freedmens Schools District 5 Day 5 Night 1 Sabbath 3 3 . Location of Schools Two Concord , Poplar Tent , Bethpage , Mt Pleasant , 2 Ms North of town . 4 . Whole number of Teachers 6 White 2 Colored 4 5 . Names postoffice address of DaySchool Teachers Mrs. J. E. Dorland W. S. Dorland , Concord , N. C. Mrs. Anna Amos Concord , N. C. Mrs. Benedict Irons Concord , N. C. John S. Harris Concord , N. C. Mr. J. W. Adams Mt . Pleasant N. C. 6 . Whole number of Schoolhouses Freedmen district 5 Their condition , capacity , value , owned By Freedmen Adequate wants of people . 7 . Number of visits Schools Day Night Sabbath 8 . Number of educational meetings held month Where 9 . Number names of places , destitute , DaySchools might organized In almost every neighborhood But particularly Rocky River ,

**!!**

In [62]:
clustered_sentences_2[751][:]

['SCHOOL REPORT OF Richmond COUNTY OF Fort Bend For Month of April 1867 . FORWARDED ... ... 1867 . RECEIVED O.S . May 8th 1867 . Mr G.C . Booth . Richmond . Texas . School Report April . 1867 .',
 'SCHOOL REPORT OF ... COUNTY OF Fort Bend , Texas For month of May 1867 FORWARDED 3rd June 1867 . RECEIVED O.S . June 5th 1867 .',
 'SCHOOL REPORT OF Richmond COUNTY OF Fort Bend For month of May 1867 FORWARDED May 31st 1867 RECEIVED O.S . ... ... .. 1867',
 'SCHOOL REPORT OF Richmond COUNTY OF Fort Bend For month of June 1867 Forwarded ... 1867 . RECEIVED O.S . July 1st of 1867 .',
 'SCHOOL REPORT OF Mrs Mary J. Cole COUNTY OF Fort Bend For month of June FORWARDED August 28 1867 . RECEIVED O.S . Aug 29th 1867 .',
 'March 67 SCHOOL REPORT OF Richmond COUNTY OF Fort Bend For month of July 1867 FORWARDED ... . 1867 . RECEIVED O.S . Sept 13 1867 .',
 'SCHOOL REPORT OF Richmond COUNTY OF Fort Bend For month of October , 1867 . FORWARDED Nov 3 1867 . RECEIVED O.S . Nov 15th 1867 . Teacher A.M.A.S 

****

# all-MiniLM-L6-v2 , threshold=3

In [63]:
start=time.time()
# Normalize the embeddings to unit length
embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=3) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddings)
cluster_assignment3 = clustering_model.labels_

clustered_sentences3 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment3):
    if cluster_id not in clustered_sentences3:
        clustered_sentences3[cluster_id] = []

    clustered_sentences3[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

242.66880416870117


In [64]:
len(clustered_sentences3)

407

In [65]:
dic3=num_of_element(cluster_assignment3,clustered_sentences3)

In [66]:
dic3[0]

92

****

# all_mpnet_base_v2 , threshold=2.5

In [67]:
start=time.time()
# Normalize the embeddings to unit length
embeddingss = embeddingss /  np.linalg.norm(embeddingss, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=2.5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddingss)
cluster_assignment_2_5 = clustering_model.labels_

clustered_sentences_2_5 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment_2_5):
    if cluster_id not in clustered_sentences_2_5:
        clustered_sentences_2_5[cluster_id] = []

    clustered_sentences_2_5[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)


360.3316855430603


In [68]:
len(clustered_sentences_2_5)

472

In [69]:
dic_2_5=num_of_element(cluster_assignment_2_5,clustered_sentences_2_5)
l_2_5=[dic_2_5[i] for i in range(len(clustered_sentences_2_5))]
maxx=np.max(l_2_5)
minn=np.min(l_2_5)
maxx, minn

(622, 8)

In [70]:
Max_clus_size(l_2_5, len(l_2_5))

[622, 560, 326, 308, 300, 291, 285, 267, 267, 266, 261, 258, 254, 243, 235, 234, 225, 225, 223, 220, 209, 207, 206, 204, 195, 193, 191, 179, 178, 176, 176, 173, 168, 167, 164, 161, 160, 156, 153, 152, 152, 151, 151, 151, 151, 150, 150, 149, 146, 143, 141, 140, 140, 140, 139, 139, 138, 136, 136, 135, 134, 134, 133, 132, 128, 127, 127, 126, 124, 123, 122, 121, 120, 119, 118, 118, 117, 117, 114, 114, 113, 113, 113, 112, 112, 111, 110, 110, 109, 109, 109, 109, 109, 108, 107, 107, 106, 106, 106, 103, 103, 102, 102, 102, 101, 100, 100, 99, 99, 98, 98, 97, 97, 97, 96, 96, 96, 96, 96, 95, 95, 94, 94, 93, 93, 93, 91, 91, 91, 91, 91, 89, 89, 89, 88, 87, 87, 86, 86, 85, 84, 84, 83, 83, 83, 82, 82, 82, 81, 80, 80, 80, 80, 79, 79, 79, 78, 78, 77, 77, 76, 76, 75, 75, 74, 74, 74, 74, 74, 73, 72, 72, 71, 71, 71, 69, 69, 69, 69, 69, 69, 69, 68, 68, 68, 68, 68, 67, 67, 67, 66, 66, 66, 65, 65, 65, 65, 64, 64, 64, 64, 64, 64, 64, 64, 64, 63, 62, 62, 62, 61, 61, 61, 61, 61, 61, 60, 60, 60, 60, 60, 60, 59, 

In [71]:
clustered_sentences_2_5[280]

['4040',
 '4041',
 '4042',
 '4043',
 '4044',
 '4045',
 '4046',
 '4047',
 '4048',
 '4049']

In [72]:
clustered_sentences_2_5[24][621]

'REPORT OF SCHOOLS FOR FREEDMEN , In City of Indianola , Texas , For month ending July 31st , 1870 NIGHT SCHOOL . MALES . FEMALES . TOTAL . No . pupils enrled last Rep Left Admitted 10 8 18 Total enrollment 10 8 18 Average attendance 10 8 18 No . paid tuition 10 4 14 No . white pupils No . always present 10 8 18 No . always punctual 8 6 4 No . 16 years of age 10 8 18 Alphabet 1 1 Primer First Reader 2 2 4 Secd advanced Rders 7 6 13 Geography Arithmetic 8 6 14 Grammar History Writing 8 6 14 No . needlework No . free war DAY SCHOOL . BOYS . GIRLS . TOTAL . No . pupils enrled last Rep Left Admitted 28 18 46 Total enrollment 28 18 46 Average attendance 24 16 40 No . paid tuition 12 5 17 No . white pupils No . always present 26 16 42 No . always punctual 23 12 35 No . 16 years of age 3 3 Alphabet 4 4 8 Primer First Reader 6 2 8 Secd advanced Rders 18 12 30 Geography 1 5 6 Arithmetic 18 12 30 Grammar History 1 3 4 Writing 18 12 30 No . needlework No . free war Sunday School . Total attendanc

In [73]:
clustered_record_2_5=pd.DataFrame({"cluster_id":cluster_assignment_2_5, "records":all_text})

In [74]:
clustered_record_2_5.head()

,cluster_id,records
0,88,one hundred dollars . I never heard Bagby comp...
1,74,The Freedmens Bureau vs. B.M . Richardson Assa...
2,74,"The Freedmens Bureau vs. B. M. Richardson , Co..."
3,277,"one hundred dollars , two suits of clothes wou..."
4,277,"questions , I gave answer , I whipped.— All I ..."


In [75]:
clustered_record_2_5.to_csv("clustered_record_2_5.csv",index=False)


****

# all_mpnet_base_v2 , threshold=2.2

In [76]:
start=time.time()
# Normalize the embeddings to unit length
embeddingss = embeddingss /  np.linalg.norm(embeddingss, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=2.2) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddingss)
cluster_assignment_2_2 = clustering_model.labels_

clustered_sentences_2_2 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment_2_2):
    if cluster_id not in clustered_sentences_2_2:
        clustered_sentences_2_2[cluster_id] = []

    clustered_sentences_2_2[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

360.8514895439148


In [77]:
len(clustered_sentences_2_2)

624

In [78]:
dic_2_2=num_of_element(cluster_assignment_2_2,clustered_sentences_2_2)
l_2_2=[dic_2_2[i] for i in range(len(clustered_sentences_2_2))]
maxx=np.max(l_2_2)
minn=np.min(l_2_2)
maxx, minn

(369, 7)

In [79]:
Max_clus_size(l_2_2, len(l_2_2))

[369, 327, 326, 308, 291, 285, 283, 267, 266, 261, 254, 235, 235, 225, 220, 209, 206, 201, 191, 191, 179, 179, 167, 166, 164, 161, 161, 160, 153, 152, 151, 151, 150, 149, 145, 140, 138, 136, 134, 133, 132, 129, 127, 122, 119, 119, 118, 117, 117, 117, 114, 112, 111, 110, 110, 109, 109, 109, 109, 109, 108, 108, 107, 106, 106, 104, 103, 103, 103, 102, 102, 101, 99, 98, 98, 98, 98, 97, 97, 96, 96, 96, 96, 96, 96, 95, 94, 93, 92, 92, 91, 91, 91, 91, 91, 90, 90, 90, 89, 89, 89, 89, 88, 88, 88, 88, 88, 88, 87, 87, 86, 86, 85, 85, 83, 82, 82, 82, 82, 81, 80, 80, 80, 80, 79, 79, 79, 79, 78, 78, 78, 78, 77, 77, 77, 76, 75, 75, 75, 75, 74, 74, 74, 72, 72, 72, 72, 71, 71, 71, 70, 69, 69, 69, 68, 68, 68, 68, 68, 67, 67, 67, 66, 66, 66, 66, 65, 65, 65, 65, 65, 64, 64, 64, 64, 64, 64, 64, 63, 63, 63, 63, 62, 62, 62, 62, 61, 61, 61, 61, 61, 61, 61, 61, 61, 60, 60, 60, 60, 60, 60, 60, 60, 59, 59, 59, 59, 59, 59, 59, 59, 59, 58, 58, 58, 58, 57, 57, 57, 56, 56, 56, 56, 56, 55, 55, 55, 54, 54, 54, 54, 54,

**the biggest  cluster main topic : REPORT OF SCHOOL FOR FREEDMEN in every USA County**

In [80]:
clustered_sentences_2_2[95]

['REPORT OF SCHOOL FOR FREEDMEN In Richmond Ft Bend Co , Texas , For Month Ending Feb 15th , 1866 DAY SCHOOL Total No . Pupils last Rep.Admitted.Left.Total Attendance.Average Attendance.Alphabet.Spelling.Reading.Mental Arithmetic.Written Arithmetic.Writing Copy Book.Writing Slate Blk.Bd.Geography.Grammar . BOYS , none3533525151500001000 GIRLS , none252253020100000700 TOTAL , 6056055352500001700 NIGHT SCHOOL Total No . Pupils last Rep.Admitted.Left.Total Attendance.Average Attendance.Alphabet.Spelling.Reading.Mental Arithmetic.Written Arithmetic.Writing Copy Book.Writing Slate Blk.Bd.Geography.Grammar . MALES , none250252020100000700 FEMALES , none290292010140000800 TOTAL , 5405440302400001500 SUNDAY SCHOOL Total attendance of Adults , 40 Total attendance of Children , 30 I certify Report correct , Regulations complied . C.E . Coleman , Teacher',
 'REPORT OF SCHOOL FOR FREEDMEN In Richmond Ft Bend Co , Texas , For Month Ending March 15th , 1866 DAY SCHOOL Total No . Pupils last Rep.Admi

**the 2nd biggest cluster main topic : REPORT OF SCHOOL FOR FREEDMEN in every USA County**

In [81]:
clustered_sentences_2_2[388]

['REPORT OF SCHOOL FOR FREEDMEN In County of Brazoria , Texas , For Month Ending May 30th , 1866 DAY SCHOOL Total No . Pupils last Rep.Admitted.Left.Total Attendance.Average Attendance.Alphabet.Spelling.Reading.Mental Arithmetic.Written Arithmetic.Writing Copy Book.Writing Slate Blk.Bd.Geography.Grammar . BOYS,194804831102018 18 GIRLS,29 TOTAL , NIGHT SCHOOL Total No . Pupils last Rep.Admitted.Left.Total Attendance.Average Attendance.Alphabet.Spelling.Reading.Mental Arithmetic.Written Arithmetic.Writing Copy Book.Writing Slate Blk.Bd.Geography.Grammar . MALES,253913929592525 2525 FEMALES,14 TOTAL , SUNDAY SCHOOL Total attendance of Adults , 27 Total attendance of Children , 45 I certify Report correct , Regulations complied . Emma C. Appelton , Teacher',
 'REPORT OF SCHOOL FOR FREEDMEN In Brazos County , Texas , For Month Ending February 28th , 1866 DAY SCHOOL Total No . Pupils last Rep.Admitted.Left.Total Attendance.Average Attendance.Alphabet.Spelling.Reading.Mental Arithmetic.Writte

**the 3nd biggest cluster main topic : Different Orders**

In [82]:
clustered_sentences_2_2[167]

['A. Q. M. Office , D. C. Bureau of R. F. A. L. Washington , D. C. Jun 24 1867 Transportation Furnished To New Rush , N. Y. Bt . Maj. J. M. Brown . A. Q. M. War Department , Bureau Refugees , Freedmen , Abandoned Lands , Office Chief Quartermaster , Washington , June 22 1867 . No . 1191 Bvt . Major J. M. Brown , A. Q. M. D. O. , Washington , D. C. Issue Transportation Mrs Eleanor . Baldwin Teacher Washington , D. C. , East Avon N. Y . By order of Maj. Gen. O. O. Howard , Commissioner . Henry M Whittelsey Bvt . Brig . Gen. , Chief Quartermaster . 668 T1 22338',
 'A. Q. M. OFFICE , D. C. Bureau of R. F. A. L. WASHINGTON , D. C. JUN 8 1867 Transportation Furnished To Annapolis Md BT MAJ. J. M. BROWN . A Q M War Department , Bureau Refugees , Freedmen , Abandoned Lands , OFFICE CHIEF QUARTERMASTER , Washington , June 8 1867 No . 768 Bvt . Major J. M. Brown , A. Q. M. D. O. , Washington , D. C. : Issue Transportation Churchill Hewitt 5 men , Destitute Freedmen , From Washington , D. C. , An

**the 4th biggest cluster main topic : Instructions regarding the education system and schools**

In [83]:
clustered_sentences_2_2[520]

['Office of Superintendent of Education , Bureau of R. , F. , A. L. STATE OF TEXAS . The observance of following REGULATIONS Is enjoined Teachers of Freedmen of Texas : 1 . The School Hours shall 9 A.M. 2:30 P.M. , half hours recess midday . 2 . The Morning Exercises commence reading of Scriptures . 3 . The three studies of primary importance Freedmen , Reading , Writing , Arithmetic . Teachers therefore make branches prominent . 4 . The Holidays shall customary White Schools of country , Discipline usual best school systems of North . 5 . Night Schools adults , Sunday Schools religious instruction , may organized Teacher . The Night Schools may kept session every week day evening , except Saturday , 7 10 oclock . 6 . Teachers allowed charge Tuition Fee of One Dollar per month ; two pupils one family , One Dollar half per month ; three one family , Two Dollars per month . 7 . A daily register of attendance kept , close of month Teacher promptly report , duplicate , SubAssistant Commiss

**the 5th biggest cluster main topic : Instructions regarding the education system and schools and notices from OFFICE SUPERINTENDENT OF SCHOOLS FOR FREEDMEN AND REFUGEES**

In [84]:
clustered_sentences_2_2[132]

['James G. Hahn Brennan April 1866 OFFICE OF SUPERINTENDENT OF SCHOOLS FOR FREEDMEN AND REFUGEES , STATE OF TEXAS . The observance of following REGULATIONS Is enjoined Teachers of Freedmen Texas',
 'James G Wahn Brennan April 1866 OFFICE OF SUPERINTENDENT OF SCHOOLS FOR FREEDMEN AND REFUGEES , STATE OF TEXAS . The observance of following REGULATIONS Is enjoined Teachers of Freedmen Texas : 1 . The School Hours shall 9 A. M. 2:30 P.M. , half hours recess midday . 2 . The Morning Exercises commence either singing of appropriate hymn , prayer , reading of Scriptures . 3 . A regular order of School Exercises shall posted School Room . 4 . The three studies of primary importance Freedmen reading , writing arithmetic . Teachers therefore make branches prominent . 5 . The Holidays shall customary white schools of State . 6 . In Freedmens Schools severity of discipline required , permitted . The modes of punishment must differ employed best school systems of North . 7 . Night Schools adults , 

**the 6th biggest cluster main topic : Documentation of property**

In [85]:
clustered_sentences_2_2[314]

['87 No . OF BILL . DISTRICT . NAMES . FOLIO OF ROLL . No . OF SQUARE . NAME OF STREET DESCRIPTION OF PROPERTY . REAL ESTATE . HORSES , MULES c. CARRIAGES , c. STEAMSHIPS , STEAMBOATS c. CAPIATL . TOTAL PERSONAL PROPERTY GRAND TOTAL . 6599485 REMARKS . 6171Burgmeyer Mrs S101290.312Franklin15001500 1500checkmark Pd Oct 21 . 6181Crepler Jacob102292.310 100100100checkmark 6191Blake John103313Basin1000X1000 1000checkmark Pd Oct 5 6201Ernst Jos.103314Basin20002000 2000checkmark 6211Bartlett Mrs Tho . J.103323Franklin10001000 1000checkmark 6221Dougherty Ellen103323Franklin10001000 1000Checkmark 6231Daurhaur H.103323Erato30003000 3000checkmark 6241Dunning Wm . Jos.103323Erato10001000 1000checkmark Pd Sept 23 6610085',
 '1 LA B R F A L 1 No . of BILL . DISTRICT . NAMES . FOLIO OF ROLL . No . of SQUARE . NAME OF STREET DESCRIPTION OF PROPERTY . REAL ESTATE . HORSES , MULES c. CARRIAGES , c. STEAMSHIPS , STEAMBOATS c. CAPIATL . TOTAL PERSONAL PROPERTY GRAND TOTAL . REMARKS . 1 1 Chase Mrs.Ann J 

**the 7th biggest cluster main topic : School issues**

In [86]:
clustered_sentences_2_2[177]

['Houston Jan 2nd 1867 E.M. Whelock Supt c Galveston Texas Dear Sir I sorry report almost every respect prospect of opening school immediately Houston unfavorable . 1st There church , excepting Methodist repared , fire . The baptist house central windows . Bureau matters unsettled , get aid . I said people morning I would become personally responsible windows stove , promise ready Monday . Miss Clay leaves work Miss Garrion wishes remain , arranged board near school . With assistance I obtain I unable get board teachers even colored families . We hotel 3.00 per day specie . Should I succeed finding board two I think of taking one Richmond . Will write fully soon Very Respectfully D.T . Allen',
 'Headquarters . Bureau of Refugees , Freedmen Abandoned Lands , State of Texas Galveston , January 10th , 1867 . Mr E. M. Whelock Supt . Schools State of Texas Sir In compliance orders recd office Dated Dec. 29th 1866 . I proceeded Houston arrange free school city , arriving Dec 30th . I found s

In [87]:
clustered_record_2_2=pd.DataFrame({"cluster_id":cluster_assignment_2_2, "records":all_text})
clustered_record_2_2.to_csv("clustered_record_2_2.csv",index=False)

****

# all-MiniLM-L6-v2 , threshold=4

In [88]:
start=time.time()
# Normalize the embeddings to unit length
embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=4) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddings)
cluster_assignment4 = clustering_model.labels_

clustered_sentences4 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment4):
    if cluster_id not in clustered_sentences4:
        clustered_sentences4[cluster_id] = []

    clustered_sentences4[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

242.95451545715332


In [89]:
len(clustered_sentences4)

237

In [90]:
dic4=num_of_element(cluster_assignment4,clustered_sentences4)

In [91]:
dic4[0]

134

****

In [92]:
start=time.time()
# Normalize the embeddings to unit length
embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddings)
cluster_assignment5 = clustering_model.labels_

clustered_sentences5 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment5):
    if cluster_id not in clustered_sentences5:
        clustered_sentences5[cluster_id] = []

    clustered_sentences5[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

241.92230796813965


In [93]:
len(clustered_sentences5)

146

In [94]:
dic5=num_of_element(cluster_assignment5,clustered_sentences5)